# Testing Stanza

# | default_exp core

In [ ]:
# !spacy download en_core_web_sm

In [ ]:
# | hide
from nbdev.showdoc import *
import project_path

# from bertopic import BERTopic
import dagshub
dagshub.init(repo_name='MeaLeon', repo_owner='AaronWChen')
# from datetime import datetime
# from hdbscan import HDBSCAN
import dill as pickle
from itertools import tee, islice
import joblib 
import json
# import matplotlib.pyplot as plt
import mlflow
from mlflow.models.signature import infer_signature
import numpy as np
import pandas as pd
import re
# from sklearn.base import TransformerMixin
# from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfTransformer,
    TfidfVectorizer,
)
from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer
# import spacy
# import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
# from spacy.lemmatizer import Lemmatizer
import stanza
from tqdm import tqdm
from typing import Any
# from umap import UMAP

# importing local scripts
# import src.custom_stanza_mlflow as stz_wrp
from src.custom_stanza_mlflow import StanzaWrapper
# import src.nlp_processor as nlpp

stanza.download('en')
nlp = stanza.Pipeline('en')

Repository initialized!

2023-11-22 22:24:55 INFO: Downloading default packages for language: en (English) ...
2023-11-22 22:24:57 INFO: File exists: /home/awchen/stanza_resources/en/default.zip
2023-11-22 22:25:03 INFO: Finished downloading models and saved to /home/awchen/stanza_resources.
2023-11-22 22:25:03 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-22 22:25:05 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

2023-11-22 22:25:05 INFO: Using device: cuda
2023-11-22 22:25:05 INFO: Loading: tokenize
2023-11-22 22:25:22 INFO: Loading: pos
2023-11-22 22:25:23 INFO: Loading: lemma
2023-11-22 22:25:24 INFO: Loading: constituency
2023-11-22 22:25:25 INFO: Loading: depparse
2023-11-22 22:25:25 INFO: Loading: sentiment
2023-11-22 22:25:26 INFO: Loading: ner
2023-11-22 22:25:28 INFO: Done loading processors!


In [ ]:
#@markdown Enter the username of your DAGsHub account:
DAGSHUB_USER_NAME = "AaronWChen" #@param {type:"string"}

#@markdown Enter the email for your DAGsHub account:
DAGSHUB_EMAIL = "awc33@cornell.edu" #@param {type:"string"}

DAGSHUB_REPO_NAME="MeaLeon"
BRANCH="STANZA-1/refactor-nltk-stanza"
dagshub.init(repo_name='MeaLeon', repo_owner='AaronWChen')

mlflow.set_tracking_uri(f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

Repository initialized!

In [ ]:
def get_experiment_id(name):
    exp = mlflow.get_experiment_by_name(name)
    if exp is None:
      exp_id = mlflow.create_experiment(name)
      return exp_id
    return exp.experiment_id

In [ ]:
def stanza_preprocessor(stanza_pipeline, ingredients_list):
    # This function takes in a Stanza pipeline and a recipe's ingredients in list form and returns a Stanza transformed document to be used in the lemmatizer
    lowered = " brk ".join(ingredients_list).lower()
    # lowered = ingredients_list.apply(" brk ".join).str.lower()
    print(type(stanza_pipeline(lowered)))
    print(stanza_pipeline(lowered))
    return stanza_pipeline(lowered)

# print(*[f'word: {word.text+" "}\t \t lemma: {word.lemma}, \t \t upos: {word.upos}' for sent in doc2.sentences for word in sent.words], sep='\n')

# This will be the tokenizer?
# lemma_test_recipe_2 = " ".join([word.lemma for sent in doc2.sentences for word in sent.words if (
#     word.upos not in ["NUM", "DET", "ADV", "CCONJ", "ADP", "SCONJ"]
# )])

In [ ]:
def stanza_lemmatizer(stanza_preprocessed):
    # This function takes in the preprocessed Stanza document from preprocessor and performs lemmatization and filtering
    
    return " ".join([word.lemma 
                      for sent in stanza_preprocessed.sentences 
                      for word in sent.words if (
                          word.upos not in ["NUM", "DET", "ADV", "CCONJ", "ADP", "SCONJ"]
                          )
                    ])
            

In [ ]:
# custom ngram analyzer function, matching only ngrams that belong to the same line
def gen_analyzer(minNgramLength, maxNgramLength):
    def ngrams_per_line(doc):

        # analyze each line of the input string seperately
        for ln in doc.split('brk'):

            # tokenize the input string (customize the regex as desired)
            terms = re.findall(u'(?u)\\b\\w+\\b', ln)

            # loop ngram creation for every number between min and max ngram length
            for ngramLength in range(minNgramLength, maxNgramLength+1):

                # find and return all ngrams
                # for ngram in zip(*[terms[i:] for i in range(3)]): <-- solution without a generator (works the same but has higher memory usage)
                for ngram in zip(*[islice(seq, i, len(terms)) for i, seq in enumerate(tee(terms, ngramLength))]): # <-- solution using a generator
                    ngram = ' '.join(ngram)
                    yield ngram
    return ngrams_per_line

In [ ]:
# custom ngram analyzer function, matching only ngrams that belong to the same line
def gen_analyzer_2(stanza_pipeline, minNgramLength, maxNgramLength):
    def ngrams_per_line(ingredients_list):

        lowered = " brk ".join(map(str, [ingred for ingred in ingredients_list if ingred is not None])).lower()
        
        if lowered is None:
            lowered = "Missing ingredients"
        
        # print("Lower case ingredients: \n")
        # print(lowered)
        # print("\n")

        preproc = stanza_pipeline(lowered)
        # print("Preprocessed ingredients: \n")
        # print(preproc)
        # print("\n")

        
        lemmad = " ".join(map(str,
                              [word.lemma 
                               for sent in preproc.sentences 
                               for word in sent.words if (
                                   word.upos not in ["NUM", "DET", "ADV", "CCONJ", "ADP", "SCONJ"]
                                #    and word not in STOP_WORDS
                                and word is not None
                            )]
                          )
                    )
        
        # print("Lemmatized ingredients: \n")
        # print(lemmad)
        # print("\n")

        # analyze each line of the input string seperately
        for ln in lemmad.split(' brk '):
            
            # tokenize the input string (customize the regex as desired)
            # terms = ln.split()
            terms = re.split("(?u)\b[a-zA-Z]{2,}\b", ln)

            # loop ngram creation for every number between min and max ngram length
            for ngramLength in range(minNgramLength, maxNgramLength+1):

                # find and return all ngrams
                # for ngram in zip(*[terms[i:] for i in range(3)]): <-- solution without a generator (works the same but has higher memory usage)
                # for ngram in zip(*[islice(seq, i, len(terms)) for i, seq in enumerate(tee(terms, ngramLength))]): # <-- solution using a generator
                for ngram in zip(*[islice(seq, i, len(terms)) for i, seq in enumerate(tee(terms, ngramLength))]): # <-- solution using a generator
                    # print("ngram: \n")
                    # print(ngram)
                    # print("\n")
                    
                    ngram = ' '.join(map(str, ngram))
                    yield ngram
                    
    return ngrams_per_line

In [ ]:
raw_data_path = '../data/recipes-en-201706/epicurious-recipes_m2.json'
food_stopwords_path = "../food_stopwords.csv"

joblib_basepath = '../joblib/2022.08.23/'

cv_path = joblib_basepath + 'countvec.joblib'
tfidf_path = joblib_basepath + 'tfidf.joblib'
full_df_path = joblib_basepath + 'recipes_with_cv.joblib'
reduced_df_path = joblib_basepath + 'reduced_df.joblib'

In [ ]:
# this is a redeem for variable naming mixed with a free pun-ish me daddy, flushtrated will be the list of all stopword to exclude so named because we're throwing these words down the drain

flushtrated = {x for x in pd.read_csv(food_stopwords_path)}
additional_to_exclude = {
    "red",
    "green",
    "black",
    "yellow",
    "white",
    "inch",
    "mince",
    "chop",
    "fry",
    "trim",
    "flat",
    "beat",
    "brown",
    "golden",
    "balsamic",
    "halve",
    "blue",
    "divide",
    "trim",
    "unbleache",
    "granulate",
    "Frank",
    "alternative",
    "american",
    "annie",
    "asian",
    "balance",
    "band",
    "barrel",
    "bay",
    "bayou",
    "beam",
    "beard",
    "bell",
    "betty",
    "bird",
    "blast",
    "bob",
    "bone",
    "breyers",
    "calore",
    "carb",
    "card",
    "chachere",
    "change",
    "circle",
    "coffee",
    "coil",
    "country",
    "cow",
    "crack",
    "cracker",
    "crocker",
    "crystal",
    "dean",
    "degree",
    "deluxe",
    "direction",
    "duncan",
    "earth",
    "eggland",
    "ener",
    "envelope",
    "eye",
    "fantastic",
    "far",
    "fat",
    "feather",
    "flake",
    "foot",
    "fourth",
    "frank",
    "french",
    "fusion",
    "genoa",
    "genovese",
    "germain",
    "giada",
    "gold",
    "granule",
    "greek",
    "hamburger",
    "helper",
    "herbe",
    "hines",
    "hodgson",
    "hunt",
    "instruction",
    "interval",
    "italianstyle",
    "jim",
    "jimmy",
    "kellogg",
    "lagrille",
    "lake",
    "land",
    "laurentiis",
    "lawry",
    "lipton",
    "litre",
    "ll",
    "maid",
    "malt",
    "mate",
    "mayer",
    "meal",
    "medal",
    "medallion",
    "member",
    "mexicanstyle",
    "monte",
    "mori",
    "nest",
    "nu",
    "oounce",
    "oscar",
    "ox",
    "paso",
    "pasta",
    "patty",
    "petal",
    "pinche",
    "preserve",
    "quartere",
    "ranch",
    "ranchstyle",
    "rasher",
    "redhot",
    "resemble",
    "rice",
    "ro",
    "roni",
    "scissor",
    "scrap",
    "secret",
    "semicircle",
    "shard",
    "shear",
    "sixth",
    "sliver",
    "smucker",
    "snicker",
    "source",
    "spot",
    "state",
    "strand",
    "sun",
    "supreme",
    "tablepoon",
    "tail",
    "target",
    "tm",
    "tong",
    "toothpick",
    "triangle",
    "trimming",
    "tweezer",
    "valley",
    "vay",
    "wise",
    "wishbone",
    "wrapper",
    "yoplait",
    "ziploc",
}

flushtrated = flushtrated.union(STOP_WORDS)
flushtrated = flushtrated.union(additional_to_exclude)
flushtrated_list = list(flushtrated)

In [ ]:
full_df = joblib.load(full_df_path)
full_df

,id,dek,hed,aggregateRating,ingredients,prepSteps,reviewsCount,willMakeAgainPct,cuisine_name,photo_filename,...,zest pith,zest vegetable,zinfandel,ziti,zucchini,zucchini blossom,zucchini crookneck,zucchini squash,árbol,árbol pepper
0,54a2b6b019925f464b373351,How does fried chicken achieve No. 1 status? B...,Pickle-Brined Fried Chicken,3.11,"[1 tablespoons yellow mustard seeds, 1 tablesp...",[Toast mustard and coriander seeds in a dry me...,7,100,Missing Cuisine,51247610_fried-chicken_1x1.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,54a408a019925f464b3733bc,Spinaci all'Ebraica,Spinach Jewish Style,3.22,"[3 pounds small-leaved bulk spinach, Salt, 1/2...",[Remove the stems and roots from the spinach. ...,5,80,Italian,EP_12162015_placeholders_rustic.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,54a408a26529d92b2c003631,"This majestic, moist, and richly spiced honey ...",New Year’s Honey Cake,3.62,"[3 1/2 cups all-purpose flour, 1 tablespoon ba...",[I like this cake best baked in a 9-inch angel...,105,88,Kosher,EP_09022015_honeycake-2.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,54a408a66529d92b2c003638,The idea for this sandwich came to me when my ...,The B.L.A.Bagel with Lox and Avocado,4.00,"[1 small ripe avocado, preferably Hass (see No...","[A short time before serving, mash avocado and...",7,100,Kosher,EP_12162015_placeholders_casual.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,54a408a719925f464b3733cc,"In 1930, Simon Agranat, the chief justice of t...",Shakshuka a la Doktor Shakshuka,2.71,"[2 pounds fresh tomatoes, unpeeled and cut in ...","[1. Place the tomatoes, garlic, salt, paprika,...",7,83,Kosher,EP_12162015_placeholders_formal.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34751,59541a31bff3052847ae2107,Buttering the bread before you waffle it ensur...,Waffled Ham and Cheese Melt with Maple Butter,0.00,"[1 tablespoon unsalted butter, at room tempera...","[Preheat the waffle iron on low., Spread a thi...",0,0,Missing Cuisine,waffle-ham-and-cheese-melt-062817.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34752,5954233ad52ca90dc28200e7,"Spread this easy compound butter on waffles, p...",Maple Butter,0.00,"[8 tablespoons (1 stick) salted butter, at roo...",[Combine the ingredients in a medium-size bowl...,0,0,Missing Cuisine,EP_12162015_placeholders_bright.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34753,595424c2109c972493636f83,Leftover mac and cheese is not exactly one of ...,Waffled Macaroni and Cheese,0.00,"[3 tablespoons unsalted butter, plus more for ...",[Preheat the oven to 375°F. Butter a 9x5-inch ...,0,0,Missing Cuisine,waffle-mac-n-cheese-062816.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34754,5956638625dc3d1d829b7166,A classic Mexican beer cocktail you can sip al...,Classic Michelada,0.00,"[Coarse salt, 2 lime wedges, 2 ounces tomato j...",[Place about 1/4 cup salt on a small plate. Ru...,0,0,Missing Cuisine,Classic Michelada 07292017.jpg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
full_df['ingredients'][0]

In [ ]:
test_recipe = ". ".join(full_df['ingredients'][0]).lower()
test_recipe

In [ ]:
doc = nlp(test_recipe)

In [ ]:
for i, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

In [ ]:
print([sentence.text for sentence in doc.sentences])


In [ ]:
print(*[f'word: {word.text+" "}\t \t lemma: {word.lemma}, \t \t upos: {word.upos}' for sent in doc.sentences for word in sent.words], sep='\n')


In [ ]:
print(*[f'lemma: {word.lemma}' for sent in doc.sentences for word in sent.words if (
    word.upos not in ["NUM", "DET", "ADV", "CCONJ", "ADP", "PUNCT", "SCONJ"]
)], sep='\n')


In [ ]:
[word.lemma for sent in doc.sentences for word in sent.words if (
    word.upos not in ["NUM", "DET", "ADV", "CCONJ", "ADP", "PUNCT", "SCONJ"]
)]

In [ ]:
recipe = []
for sent in doc.sentences:
    ingredients = []
    for word in sent.words:
        if word.upos not in ["NUM", "DET", "ADV", "CCONJ", "ADP", "PUNCT", "SCONJ"]:
            ingredients.append(word.lemma)
        else:
            pass
    recipe.append(ingredients)

recipe

Found [this resource](https://stackoverflow.com/questions/26907309/create-ngrams-only-for-words-on-the-same-line-disregarding-line-breaks-with-sc), trying custom analyzer

In [ ]:
# this is probably going to be the preprocessor
test_recipe_2 = " brk ".join(full_df['ingredients'][0]).lower()
doc2 = nlp(test_recipe_2)

# print(*[f'word: {word.text+" "}\t \t lemma: {word.lemma}, \t \t upos: {word.upos}' for sent in doc2.sentences for word in sent.words], sep='\n')

# This will be the tokenizer?
# lemma_test_recipe_2 = " ".join([word.lemma for sent in doc2.sentences for word in sent.words if (
#     word.upos not in ["NUM", "DET", "ADV", "CCONJ", "ADP", "SCONJ"]
# )])
lemma_test_recipe_2 = [" ".join([word.lemma for sent in doc2.sentences for word in sent.words if (
    word.upos not in ["NUM", "DET", "ADV", "CCONJ", "ADP", "SCONJ"]
)])]
lemma_test_recipe_2

In [ ]:
cv = CountVectorizer(analyzer=gen_analyzer(1, 4))
cv.fit(lemma_test_recipe_2)

In [ ]:
X = cv.fit_transform(lemma_test_recipe_2)
cv.get_feature_names_out()

In [ ]:
test_recipe_pipeline = full_df['ingredients'][0]

cv_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'preprocessor':stanza_preprocessor,
    'tokenizer':stanza_lemmatizer, # out of memory 
    # 'stop_words':flushtrated_list,
    'analyzer': gen_analyzer(1, 4),
    'min_df':10,
}

vectorizer_model = CountVectorizer(**cv_params)

test_transform = vectorizer_model.fit_transform(test_recipe_pipeline)

In [ ]:
test_recipe_pipeline = full_df['ingredients'][0]

test_recipe_preproc = stanza_preprocessor(nlp, test_recipe_pipeline)

In [ ]:
test_recipe_lemma = stanza_lemmatizer(test_recipe_preproc)
test_recipe_lemma

In [ ]:
for ln in test_recipe_lemma.split('brk'):

    # tokenize the input string (customize the regex as desired)
    terms = re.findall(u'(?u)\\b\\w+\\b', ln)

    # loop ngram creation for every number between min and max ngram length
    for ngramLength in range(1, 5):

        # find and return all ngrams
        for ngram in zip(*[terms[i:] for i in range(3)]): #<-- solution without a generator (works the same but has higher memory usage)
        # for ngram in zip(*[islice(seq, i, len(terms)) for i, seq in enumerate(tee(terms, ngramLength))]): # <-- solution using a generator
            ngram = ' '.join(ngram)
            # yield ngram
ngram


In [ ]:
test_recipe_lemma

In [ ]:
test_recipe_lemma.split(' brk ')

In [ ]:
gen_analyzer_tester(1,4)

In [ ]:
temp = full_df["ingredients"][0:50].apply("|".join)


cv_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'preprocessor':stanza_preprocessor,
    'tokenizer':stanza_lemmatizer, # out of memory 
    # 'stop_words':flushtrated_list,
    'analyzer': gen_analyzer(1, 4),
    'min_df':10,
}

vectorizer_model = CountVectorizer(**cv_params)

test_transform = vectorizer_model.fit_transform(temp)#:5])
vectorizer_model.get_feature_names_out()

based on the warnings (preprocessor not used since analyzer is callable and tokenizer not used since analyzer is not word), may need to move preprocessing into analyzer. And based on [this](https://stackoverflow.com/questions/63185843/scikit-learn-countvectorizer-customizing-preprocessor-tokenizer-and-analyzer), will need to incorporate stopwords into the analyzer

In [ ]:
# temp = full_df["ingredients"][0:500]#.apply("|".join)

cv_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': gen_analyzer_2(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

vectorizer_model = CountVectorizer(**cv_params)

test_transform = vectorizer_model.fit_transform(tqdm(full_df["ingredients"]))
# test_transform = vectorizer_model.fit_transform(temp)
vectorizer_model.get_feature_names_out()

In [ ]:
vectorizer_model.transform(full_df["ingredients"][1200])

In [ ]:
cv_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': gen_analyzer_2(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

tfidf_vectorizer_model = TfidfVectorizer(**cv_params)

test_tfidf_transform = tfidf_vectorizer_model.fit_transform(tqdm(full_df["ingredients"]))
# test_transform = vectorizer_model.fit_transform(temp)
# tfidf_vectorizer_model.get_feature_names_out()


In [ ]:
word_matrix = pd.DataFrame(
        test_tfidf_transform.toarray(), columns=tfidf_vectorizer_model.get_feature_names_out(), index=full_df.index
    )

with open("../joblib/tfidf_transformer.joblib", "wb") as fo:
          joblib.dump(tfidf_vectorizer_model, fo, compress=True)

with open("../joblib/database_word_matrix.joblib", "wb") as fo:
          joblib.dump(word_matrix, fo, compress=True)

In [ ]:
# stz_wrpper = stz_wrp.StanzaWrapper()

cv_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
    'min_df':10,
}

tfidf_vectorizer_model = TfidfVectorizer(**cv_params)

test_tfidf_transform = tfidf_vectorizer_model.fit_transform(tqdm(full_df["ingredients"]))

word_matrix = pd.DataFrame(
        test_tfidf_transform.toarray(), columns=tfidf_vectorizer_model.get_feature_names_out(), index=full_df["ingredients"].index
    )

with open("../joblib/tfidf_transformer.pkl", "wb") as fo:
          pickle.dump(tfidf_vectorizer_model, fo)

with open("../joblib/database_word_matrix.pkl", "wb") as fo:
          pickle.dump(word_matrix, fo)

100%|██████████| 34656/34656 [6:35:04<00:00,  1.46it/s]   


In [ ]:
with mlflow.start_run(experiment_id=get_experiment_id("stanza_quadgrams_small_set_v1")):
    cv_params = {
        'strip_accents':"unicode",
        'lowercase':True,
        'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
        'min_df':10,
    }

    pipeline_params = {
        'stanza_model': 'en',
        'language': 'english',
        'sklearn-transformer': 'TfidfVectorizer'
    }

    pipeline_params.update(cv_params)

    mlflow.log_params(pipeline_params)

    tfidf_vectorizer_model = TfidfVectorizer(**cv_params)

    test_tfidf_transform = tfidf_vectorizer_model.fit_transform(tqdm(full_df["ingredients"][0:50]))

    word_matrix = pd.DataFrame(
            test_tfidf_transform.toarray(), columns=tfidf_vectorizer_model.get_feature_names_out(), index=full_df["ingredients"][0:50].index
        )

    with open("../joblib/tfidf_transformer_small_test.pkl", "wb") as fo:
            pickle.dump(tfidf_vectorizer_model, fo)
            # mlflow.log_artifact("../joblib/database_word_matrix.pkl")

    with open("../joblib/database_word_matrix_small_test.pkl", "wb") as fo:
            pickle.dump(word_matrix, fo)
            # mlflow.log_artifact("../joblib/database_word_matrix.pkl")

    mlflow.log_artifacts("../joblib/", artifact_path="sklearn_dill_pkls")

100%|██████████| 50/50 [00:43<00:00,  1.14it/s]


In [ ]:
with mlflow.start_run(experiment_id=get_experiment_id("stanza_quadgrams_v1")):
    cv_params = {
        'strip_accents':"unicode",
        'lowercase':True,
        'analyzer': StanzaWrapper().stanza_analyzer(stanza_pipeline=nlp, minNgramLength=1, maxNgramLength=4),
        'min_df':10,
    }

    pipeline_params = {
        'stanza_model': 'en',
        'language': 'english',
        'sklearn-transformer': 'TfidfVectorizer'
    }

    pipeline_params.update(cv_params)

    mlflow.log_params(pipeline_params)

    # tfidf_vectorizer_model = TfidfVectorizer(**cv_params)

    # test_tfidf_transform = tfidf_vectorizer_model.fit_transform(tqdm(full_df["ingredients"]))

    # word_matrix = pd.DataFrame(
    #         test_tfidf_transform.toarray(), columns=tfidf_vectorizer_model.get_feature_names_out(), index=full_df["ingredients"].index
    #     )

    # with open("../joblib/tfidf_transformer.pkl", "wb") as fo:
    #         pickle.dump(tfidf_vectorizer_model, fo)
    #         # mlflow.log_artifact("../joblib/database_word_matrix.pkl")

    # with open("../joblib/database_word_matrix.pkl", "wb") as fo:
    #         pickle.dump(word_matrix, fo)
    #         # mlflow.log_artifact("../joblib/database_word_matrix.pkl")

    mlflow.log_artifacts("../joblib/", artifact_path="sklearn_dill_pkls")

In [ ]:
# was getting an error (TypeError: sequence item 37: expected str instance, NoneType found) saying that some recipes have NoneType, and wondered if there were recipes with no ingredients for some reason
full_df[full_df["ingredients"].isna()]

In [ ]:
full_df["ingredients"][34:40]

In [ ]:
subset = full_df['ingredients'][0:5].apply(" brk ".join).str.lower()
subset


In [ ]:
test_preproc = full_df['ingredients'][0:5].apply(stanza_preprocessor, args=(nlp,))
test_preproc

In [ ]:
preproc = stanza_preprocessor(nlp, full_df['ingredients'][0:5][0])
preproc

In [ ]:
lemmad = stanza_lemmatizer(preproc)
lemmad

In [ ]:
full_df['prepSteps']

In [ ]:
full_df['prepSteps'][0]

In [ ]:
recipe_steps = full_df['prepSteps'].apply(" ".join).str.lower()

In [ ]:
recipe_steps

In [ ]:
# pre-calculating sentence mebeddings
# embedding_model_params = {'embedding_model': 'all-MiniLM-L6-v2'}
# embedding_model = SentenceTransformer(embedding_model_params['embedding_model'])
# embeddings = embedding_model.encode(recipe_steps, show_progress_bar=True)

In [ ]:
# specify UMAP dimensionality reductions
# umap_model_params = {'n_neighbors':15, 'n_components':10, 'random_state':200}
# umap_model = UMAP(**umap_model_params)

In [ ]:
# cluster with HDBSCAN
# hdbscan_model_params = {'min_cluster_size':200, 'prediction_data':True}
# hdbscan_model = HDBSCAN(**hdbscan_model_params)

In [ ]:
# adding custom count vectorization
cv_params = {
    'strip_accents':"unicode",
    'lowercase':True,
    # 'preprocessor':custom_preprocessor,
    # 'tokenizer':custom_lemmatizer, # out of memory 
    'stop_words':flushtrated_list,
    'token_pattern':r"(?u)\b[a-zA-Z]{2,}\b",
    'ngram_range':(1, 4),
    'min_df':10,
}

vectorizer_model = CountVectorizer(**cv_params)

In [ ]:
# # with open('../data/processed/bertopic_params.joblib', 'w') as fp:
# pipeline_params = {
#     'embedding':{'pretrained_sentence_embeddings': embedding_model_params},
#     'dimension_reduction': {'UMAP': umap_model_params},
#     'clustering': {'HDBSCAN': hdbscan_model_params},
#     'vectorizer': {'sklearn_countvectorizer': cv_params},
# }
# joblib.dump(pipeline_params, '../data/processed/bertopic_params.joblib')

In [ ]:
with mlflow.start_run(experiment_id=get_experiment_id("initial_explicit_spec_run_3")):
    pipeline_params = {
        'language':'english',
        'top_n_words':20,
        'n_gram_range':(1, 4),
        'min_topic_size':500,
        'nr_topics':'auto',
        'verbose':True,
        'low_memory':True,
        'calculate_probabilities':True
    }
    mlflow.log_params(pipeline_params)
    
    topic_model = BERTopic(
        **pipeline_params,
        vectorizer_model=vectorizer_model
    )
    # TOKENIZERS_PARALLELISM=False

    topics, probs = topic_model.fit_transform(recipe_steps)

    topic_model.get_topic_info().to_json('../data/processed/topic_model_df.json')

    # mlflow.log_artifact('../data/processed/bertopic_params.joblib')
    mlflow.log_artifact('../data/processed/topic_model_df.json')

    print(topic_model.get_topic_info())

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
print_full(topic_model.get_topic_info())

In [ ]:
topic_model.get_topic_info()['Representation']

# | export
def foo():
    pass

In [ ]:
# | hide
# import nbdev

# nbdev.nbdev_export()